### Import Required Libraries and Set Up Environment Variables

In [236]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [237]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [238]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build the NYT URL
# query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
base_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&fl={field_list}"


In [239]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{base_url}&page={str(page)}"    
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    # print(reviews)

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f'Checked page {page}')

    except:
        # Print the page number that had no results then break from the loop
        print(f"No results for page number {page}")
        break

print(f"Found {len(reviews_list)} matching reviews")

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
Found 200 matching reviews


In [240]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/06/20/movies/sublime-review.html",
        "snippet": "A teenager dreams of pop songs, and his best friend, in Mariano Biasin\u2019s tender gay coming-of-age drama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Sublime\u2019 Review: Two Boys, One in Love",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Sublime",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Sublime (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Biasin, Mariano",
         

In [241]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = pd.json_normalize(reviews_list)
nyt_reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/06/20/movies/subl...,"A teenager dreams of pop songs, and his best f...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-06-20T17:48:07+0000,266,"‘Sublime’ Review: Two Boys, One in Love",None,None,Sublime,None,None,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None
1,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,"[{'name': 'creative_works', 'value': 'Kangaroo...",2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
2,https://www.nytimes.com/1991/10/11/movies/shor...,"Short-order cookery, dreams of love. Warm and ...",The New York Times,"[{'name': 'subject', 'value': 'MOTION PICTURES...",1991-10-11T05:00:00+0000,1117,Short-Order Cookery And Dreams of Love,None,None,Short-Order Cookery And Dreams of Love,None,None,None,By Janet Maslin,"[{'firstname': 'Janet', 'middlename': None, 'l...",None
3,https://www.nytimes.com/1989/07/12/movies/revi...,Rob Reiner’s take on romantically bruised New ...,The New York Times,"[{'name': 'subject', 'value': 'Reviews', 'rank...",1989-07-12T05:00:00+0000,942,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,None,By Caryn James,"[{'firstname': 'Caryn', 'middlename': None, 'l...",None
4,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None


In [242]:
nyt_reviews_df.value_counts("headline.main")

headline.main
'ELENI' ENSHRINES ON FILM A MOTHER'S LEGACY OF LOVE                         1
Stealing Love and Loyalty, Among Some Other Things                          1
Review: ‘Touched With Fire,’ a Love Story Between Two Bipolar Poets         1
Reviews/Film; Of Butchers and Love                                          1
SCREEN: 'SPLASH,' A MERMAID'S LOVE                                          1
                                                                           ..
Love, Sex, Desire and Lyrics (Oh, and Stiletto Pumps) Across the Decades    1
Loved Ones                                                                  1
Love’s Eternal Masquerade                                                   1
Make a Date With Love                                                       1
‘Your Name Engraved Herein’ Review: When Love Is All You Can See            1
Name: count, Length: 200, dtype: int64

In [243]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


# Define the refined regex pattern
pattern = r'\u2018([^"]*)\u2019\s*Review'

# Apply the regex to each cell to extract the movie title
nyt_reviews_df['title'] = nyt_reviews_df['headline.main'].apply(lambda x: re.search(pattern, x).group(0) if re.search(pattern, x) else x)
print(nyt_reviews_df['title'])

0                                       ‘Sublime’ Review
1      Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...
2                 Short-Order Cookery And Dreams of Love
3      Review/Film; It's Harry  [ Loves ]  Sally in a...
4                              Love’s Eternal Masquerade
                             ...                        
195    A Paragon of Honor, With a Little Time Off for...
196    Where Hearts Don't Cheat, And Love Is the Current
197    Review: In ‘Thelma,’ a Woman in Love Can Burn ...
198                    Theft, Adultery and Good Old Love
199          Family Values and Love In Vietnam Aftermath
Name: title, Length: 200, dtype: object


In [244]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df['keywords'] = nyt_reviews_df['keywords'].apply(extract_keywords)


In [245]:
nyt_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/06/20/movies/subl...,"A teenager dreams of pop songs, and his best f...",The New York Times,subject: Movies;creative_works: Sublime (Movie...,2023-06-20T17:48:07+0000,266,"‘Sublime’ Review: Two Boys, One in Love",None,None,Sublime,None,None,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None,‘Sublime’ Review
1,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...
2,https://www.nytimes.com/1991/10/11/movies/shor...,"Short-order cookery, dreams of love. Warm and ...",The New York Times,subject: MOTION PICTURES;subject: REVIEW;creat...,1991-10-11T05:00:00+0000,1117,Short-Order Cookery And Dreams of Love,None,None,Short-Order Cookery And Dreams of Love,None,None,None,By Janet Maslin,"[{'firstname': 'Janet', 'middlename': None, 'l...",None,Short-Order Cookery And Dreams of Love
3,https://www.nytimes.com/1989/07/12/movies/revi...,Rob Reiner’s take on romantically bruised New ...,The New York Times,subject: Reviews;subject: MOTION PICTURES;crea...,1989-07-12T05:00:00+0000,942,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,None,By Caryn James,"[{'firstname': 'Caryn', 'middlename': None, 'l...",None,Review/Film; It's Harry [ Loves ] Sally in a...
4,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerade


In [246]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = nyt_reviews_df['title'].to_list()
title_list

['‘Sublime’ Review',
 'Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre',
 'Short-Order Cookery And Dreams of Love',
 "Review/Film; It's Harry  [ Loves ]  Sally in a Romance Of New Yorkers and Neuroses",
 'Love’s Eternal Masquerade',
 '‘The Tomorrow Man’ Review',
 'FILM REVIEW;Equal Opportunity in Games of Love',
 '‘Marriage Story’ Review',
 '‘Bones and All’ Review',
 '‘Love, Antosha’ Review',
 '‘Molli and Max in the Future’ Review',
 '‘You Can Live Forever’ Review',
 'From Tapestried Villa to Sylvan Glade, Aristocratic Women in Love',
 "A Love Triangle Like No Other in 'Two Drifters' ",
 'Teenager Dies, Leaving Love, and a Life, Behind',
 'FILM REVIEW;Of a Circus, Church and Lesbian Love',
 'Homage to a Poet Who Loved Poland',
 'Review/Film;   Love and Pain Among the Gentry in Ireland',
 'Review: ‘Prem Ratan Dhan Payo,’ a Bollywood Tale of a Prince-and-Plebe Double',
 'Review/Film; Mysticism and Love Inside a Hammer and Sickle',
 'A Promoter Finds a New Cause: God',
 

### Access The Movie Database API

In [247]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = f"&api_key={tmdb_api_key}"

In [248]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(6)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    
    tmdb_query_url = f"{tmdb_url}{title}{tmdb_key_string}"
    tmdb_titles = requests.get(tmdb_query_url).json()
    # print(json.dumps(tmdb_titles, indent=4))

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
        for movie in tmdb_titles['results']:
            title = movie['title']

        # Get movie id
        # Extract the movie id from the first result
        movie_id = tmdb_titles['results'][0]['id']
        print(f"Movie ID: {movie_id}")

        # Make a request for a the full movie details
        # Execute "GET" request with url
        movie_details = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()
        # print(json.dumps(movie_details, indent=4))
        
        # Extract the genre names into a list
        genre_names = [genre['name'] for genre in movie_details['genres']]
        # print(genre_names)

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
        # print(spoken_languages)

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]
        # print(production_countries)

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_details['title'],
            "release_date": movie_details['release_date'],
            "runtime": movie_details['runtime'],
            "genres": genre_names,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })
        
        # Print out the title that was found
        print(f"Found movie with title: {title}")

    except:
        print(f"No result for: {title}")
        pass



No result for: ‘Sublime’ Review
No result for: Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre
No result for: Short-Order Cookery And Dreams of Love
No result for: Review/Film; It's Harry  [ Loves ]  Sally in a Romance Of New Yorkers and Neuroses
No result for: Love’s Eternal Masquerade
No result for: ‘The Tomorrow Man’ Review
No result for: FILM REVIEW;Equal Opportunity in Games of Love
No result for: ‘Marriage Story’ Review
No result for: ‘Bones and All’ Review
No result for: ‘Love, Antosha’ Review
No result for: ‘Molli and Max in the Future’ Review
No result for: ‘You Can Live Forever’ Review
No result for: From Tapestried Villa to Sylvan Glade, Aristocratic Women in Love
No result for: A Love Triangle Like No Other in 'Two Drifters' 
No result for: Teenager Dies, Leaving Love, and a Life, Behind
No result for: FILM REVIEW;Of a Circus, Church and Lesbian Love
No result for: Homage to a Poet Who Loved Poland
No result for: Review/Film;   Love and Pain Among the Gent

In [249]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(movie_details, indent=4))


{
    "adult": false,
    "backdrop_path": "/8RE7gJV8tqEJ52C5dlRTWUinieO.jpg",
    "belongs_to_collection": null,
    "budget": 0,
    "genres": [
        {
            "id": 18,
            "name": "Drama"
        },
        {
            "id": 10749,
            "name": "Romance"
        }
    ],
    "homepage": "",
    "id": 71125,
    "imdb_id": "tt0089160",
    "origin_country": [
        "US"
    ],
    "original_language": "en",
    "original_title": "Fool for Love",
    "overview": "A woman is waiting in a motel for her boyfriend, when an old flame turns up and tries to take her back to the life she is trying to leave behind.",
    "popularity": 4.283,
    "poster_path": "/zZciKF71Fa96kx7UZ3nl57pgkdA.jpg",
    "production_companies": [
        {
            "id": 1444,
            "logo_path": "/1848gsFuV3EGewKg8xWADAyJUVy.png",
            "name": "The Cannon Group",
            "origin_country": "US"
        },
        {
            "id": 13549,
            "logo_path": null,

In [250]:
# Convert the results to a DataFrame
pd.json_normalize(movie_details)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/8RE7gJV8tqEJ52C5dlRTWUinieO.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,71125,tt0089160,[US],en,...,1985-12-06,0,106,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Fool for Love,False,5.78,41


### Merge and Clean the Data for Export

In [251]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_movies_df = pd.merge(nyt_reviews_df, pd.DataFrame(tmdb_movies_list), on="title", how="left")
merged_movies_df.head(20)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.person,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/06/20/movies/subl...,"A teenager dreams of pop songs, and his best f...",The New York Times,subject: Movies;creative_works: Sublime (Movie...,2023-06-20T17:48:07+0000,266,"‘Sublime’ Review: Two Boys, One in Love",None,None,Sublime,...,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None,‘Sublime’ Review,NaN,NaN,NaN,NaN,NaN
1,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,...,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,NaN,NaN,NaN,NaN,NaN
2,https://www.nytimes.com/1991/10/11/movies/shor...,"Short-order cookery, dreams of love. Warm and ...",The New York Times,subject: MOTION PICTURES;subject: REVIEW;creat...,1991-10-11T05:00:00+0000,1117,Short-Order Cookery And Dreams of Love,None,None,Short-Order Cookery And Dreams of Love,...,None,By Janet Maslin,"[{'firstname': 'Janet', 'middlename': None, 'l...",None,Short-Order Cookery And Dreams of Love,NaN,NaN,NaN,NaN,NaN
3,https://www.nytimes.com/1989/07/12/movies/revi...,Rob Reiner’s take on romantically bruised New ...,The New York Times,subject: Reviews;subject: MOTION PICTURES;crea...,1989-07-12T05:00:00+0000,942,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,Review/Film; It's Harry [ Loves ] Sally in a...,...,None,By Caryn James,"[{'firstname': 'Caryn', 'middlename': None, 'l...",None,Review/Film; It's Harry [ Loves ] Sally in a...,NaN,NaN,NaN,NaN,NaN
4,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,...,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerade,NaN,NaN,NaN,NaN,NaN
5,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,The New York Times,subject: Movies;creative_works: The Tomorrow M...,2019-05-21T11:00:01+0000,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,‘The Tomorrow Man’ Review,NaN,NaN,NaN,NaN,NaN
6,https://www.nytimes.com/1996/01/12/movies/film...,Housewife pretends to fall for lesbian. Risque...,The New York Times,subject: MOTION PICTURES;subject: Reviews;crea...,1996-01-12T05:00:00+0000,943,FILM REVIEW;Equal Opportunity in Games of Love,FILM REVIEW,None,FILM REVIEW;Equal Opportunity in Games of Love,...,None,By Janet Maslin,"[{'firstname': 'Janet', 'middlename': None, 'l...",None,FILM REVIEW;Equal Opportunity in Games of Love,NaN,NaN,NaN,NaN,NaN
7,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,The New York Times,"subject: Movies;persons: Baumbach, Noah;creati...",2019-11-05T19:01:02+0000,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",...,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,‘Marriage Story’ Review,NaN,NaN,NaN,NaN,NaN
8,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,subject: Movies;creative_works: Bones and All ...,2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",..

In [252]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

# Function to remove brackets and quotes
def strip_brackes_and_quotes(x):
    return str(x).strip('[]').replace("'", "")

# Convert tmdb_movies_list to a DataFrame
# tmdb_movies_df = pd.DataFrame(tmdb_movies_list)

# Apply the function to the 'genres' column
merged_movies_df['genres'] = merged_movies_df['genres'].apply(strip_brackes_and_quotes)

# Apply the function to the 'spoken_languages' column
merged_movies_df['spoken_languages'] = merged_movies_df['spoken_languages'].apply(strip_brackes_and_quotes)

# Apply the function to the 'production_countries' column
merged_movies_df['production_countries'] = merged_movies_df['production_countries'].apply(strip_brackes_and_quotes)

# Display the updated DataFrame
merged_movies_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.person,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/06/20/movies/subl...,"A teenager dreams of pop songs, and his best f...",The New York Times,subject: Movies;creative_works: Sublime (Movie...,2023-06-20T17:48:07+0000,266,"‘Sublime’ Review: Two Boys, One in Love",None,None,Sublime,...,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None,‘Sublime’ Review,NaN,NaN,nan,nan,nan
1,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,...,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/1991/10/11/movies/shor...,"Short-order cookery, dreams of love. Warm and ...",The New York Times,subject: MOTION PICTURES;subject: REVIEW;creat...,1991-10-11T05:00:00+0000,1117,Short-Order Cookery And Dreams of Love,None,None,Short-Order Cookery And Dreams of Love,...,None,By Janet Maslin,"[{'firstname': 'Janet', 'middlename': None, 'l...",None,Short-Order Cookery And Dreams of Love,NaN,NaN,nan,nan,nan
3,https://www.nytimes.com/1989/07/12/movies/revi...,Rob Reiner’s take on romantically bruised New ...,The New York Times,subject: Reviews;subject: MOTION PICTURES;crea...,1989-07-12T05:00:00+0000,942,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,Review/Film; It's Harry [ Loves ] Sally in a...,...,None,By Caryn James,"[{'firstname': 'Caryn', 'middlename': None, 'l...",None,Review/Film; It's Harry [ Loves ] Sally in a...,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,...,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerade,NaN,NaN,nan,nan,nan


In [253]:
# Drop "byline.person" column
merged_movies_df.drop(columns=["byline.person"], inplace=True)
merged_movies_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/06/20/movies/subl...,"A teenager dreams of pop songs, and his best f...",The New York Times,subject: Movies;creative_works: Sublime (Movie...,2023-06-20T17:48:07+0000,266,"‘Sublime’ Review: Two Boys, One in Love",None,None,Sublime,...,None,None,By Erik Piepenburg,None,‘Sublime’ Review,NaN,NaN,nan,nan,nan
1,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,...,None,None,By Ken Jaworowski,None,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/1991/10/11/movies/shor...,"Short-order cookery, dreams of love. Warm and ...",The New York Times,subject: MOTION PICTURES;subject: REVIEW;creat...,1991-10-11T05:00:00+0000,1117,Short-Order Cookery And Dreams of Love,None,None,Short-Order Cookery And Dreams of Love,...,None,None,By Janet Maslin,None,Short-Order Cookery And Dreams of Love,NaN,NaN,nan,nan,nan
3,https://www.nytimes.com/1989/07/12/movies/revi...,Rob Reiner’s take on romantically bruised New ...,The New York Times,subject: Reviews;subject: MOTION PICTURES;crea...,1989-07-12T05:00:00+0000,942,Review/Film; It's Harry [ Loves ] Sally in a...,None,None,Review/Film; It's Harry [ Loves ] Sally in a...,...,None,None,By Caryn James,None,Review/Film; It's Harry [ Loves ] Sally in a...,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,...,None,None,By David DeWitt,None,Love’s Eternal Masquerade,NaN,NaN,nan,nan,nan


In [254]:
# Delete duplicate rows and reset index
merged_movies_df.reset_index(drop=True, inplace=True)


In [255]:
# Export data to CSV without the index
merged_movies_df.to_csv("./output/merged_movie_data.csv", index=False)
